In [ ]:
import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt

In [ ]:
def read_benchmark(path):
    dfs = []

    with open(path, 'r') as file:
        for doc in yaml.safe_load_all(file):
            instance = "-".join(doc["instance"].split("/")[-1].split("-")[:-1])
            lower_bound = doc["lower_bound_name"]
            #iterations = doc["iterations"]
            #seed = experiment["seed"]
            dfs.append(pd.DataFrame([[instance, lower_bound, np.mean(doc["values"]), np.mean(doc["complete_times"]), np.std(doc["complete_times"])]], columns=["instance", "lower_bound", "value", "time_mean", "time_std"]))

    return pd.concat(dfs, ignore_index=True)

In [ ]:
paths = ["../output/lb_greedy.yaml", "../output/lb_local_search.yaml", "../output/lb_linearprogram.yaml"]
df = pd.concat(map(read_benchmark, paths)) 

In [ ]:
df["c"] = df["lower_bound"]
df.replace({"c": {"Greedy": 0, "LocalSearch": 1, "LinearProgram": 2}}, inplace=True)
df["time_mean"] = df["time_mean"] / 1e9
df["time_std"] = df["time_std"] / 1e9
df

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.scatter(df["value"], df["time_mean"], c=df["c"])
ax.set_xlabel("lower bound value")
ax.set_ylabel("time (s)")
plt.show()

In [ ]:
df.drop_duplicates(["instance", "lower_bound"], inplace=True)
a = df[df["c"] == 0]
b = df[df["c"] == 1].set_index("instance")
c = df[df["c"] == 2].set_index("instance")
df = a.join(b, on="instance", lsuffix="_greedy", rsuffix="_localsearch").join(c, on="instance", rsuffix="_linearprogram")

In [ ]:
df

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["value_greedy"], df["value"])
ax.plot(df["value_greedy"], df["value_greedy"])
plt.show()

fig, ax = plt.subplots()
ax.scatter(df["time_mean_greedy"], df["time_mean"])
ax.plot(df["time_mean_greedy"], df["time_mean_greedy"])
plt.show()

In [ ]:
plt.plot(np.sort(df["time_mean_greedy"]))
plt.plot(np.sort(df["time_mean_localsearch"]))
plt.show()

plt.scatter(df["time_mean_greedy"], df["time_mean_localsearch"])
plt.show()